In [182]:
from russtress import Accent
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import pymorphy2
from gensim.models import KeyedVectors
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
from copy import deepcopy
from itertools import permutations, product
from pprint import pprint

In [111]:
a = Accent()
morph = pymorphy2.MorphAnalyzer()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()
morph = pymorphy2.MorphAnalyzer()

В конце видно, что особых отличий на этом куске текста нет. Вторая модель быстрее загружается в тетрадке и доступна для использованя на веб-сервисах.

In [189]:
#model = KeyedVectors.load_word2vec_format('ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec')
model = KeyedVectors.load_word2vec_format('model.bin', binary=True)
#model = KeyedVectors.load_word2vec_format('araneum_upos_skipgram_300_2_2018.vec')

# Предобработка данных

Ручная:
* удалены пометы (см. § N)
* расшифрованы сокращения и числительные
* авторы примеров включены в тег примеров

In [56]:
with open (r'rusgram\Том _1\1437-1454_edited.html', 'r') as text:
    text = text.read()

In [57]:
soup = BeautifulSoup(text, markupMassage=False)
paragraphs = soup.find_all(['p','center'])

In [58]:
clean_text = []

In [59]:
for paragraph in paragraphs:
    if (paragraph.name == 'center')and(paragraph.b != None):
        paragraph_title = paragraph.find_all('b')
        paragraph_title = ["{}".format(x) for x in paragraph_title]
        paragraph_title = re.sub('<br/>', '\n', (' ').join(paragraph_title))
        paragraph_title = re.sub('<.+?>', '', paragraph_title)
        clean_text.append((paragraph_title, 'paragraph_title'))
    elif 'JUSTIFY' in paragraph.attrs.values():
        paragraph = re.sub('\xa0', ' ', str(paragraph))
        paragraph = re.sub('\xad', ' ', paragraph)
        paragraph = re.sub('<i>', ' example ', paragraph)
        paragraph = re.sub('</i>', ' endexample ', paragraph)
        paragraph = re.sub('<span style="letter-spacing: 3px">', ' term ', paragraph)
        paragraph = re.sub('</span>', ' endterm ', paragraph)
        paragraph = re.sub('<.+?>', '', paragraph)
        clean_text.append((paragraph, 'paragraph'))

In [60]:
def check_word(word, lang='ru'):
    """Функция определяет, есть ли в строке кириллица или латиница, за исключением знака j,
    так как он может обозначаться на письме в качестве аффикса"""
    word = re.sub('j', '', word)
    rus=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    en=set('qwertyuiopasdfghjklzxcvbnm')
    if lang == 'ru':
        return not rus.isdisjoint(word.lower())
    elif lang == 'en':
        return not en.isdisjoint(word.lower())
    else:
        print('Допустимые значения lang: en или rus')

In [61]:
def check_gender(tags):
    """Функция определяет, какого рода слово, чьи теги поданы на вход"""
    tags = str(tags)
    if 'masc' in tags:
        return 'masc'
    elif 'femn' in tags:
        return 'femn'
    elif 'neut' in tags:
        return 'neut'

In [62]:
def check_number(tags):
    """Функция определяет, в каком числе стоит слово, чьи теги поданы на вход"""
    tags = str(tags)
    if 'sing' in tags:
        return 'sing'
    if 'plur' in tags:
        return 'plur'

In [63]:
def putting_stress(word):
    """Функция проставляет ударение в слове, учитывая возможность,
    что слово будет подано вместе с предлогом"""
    word = word.split(' ')
    word = (' ').join(word[:len(word) - 1]) + ' ' + a.put_stress(word[-1])
    return word

Проблема: часто можно встретить слова, помеченные как примеры, перемешанными с основным текстом. Выносить одно слово из стихотворного текста как пример странно. 

Что делать: хочется иметь определенный лимит, например, если у нас встречается меньше n или равно слов-примеров подряд, оставлять их в тексте, но сохранять пометку example, так как подобные слова мы не можем заменять. Также, если у нас меньше или равно m слов основного текста между примерами, то включать слова основного текста в примеры. 

Идея: в тексте примеры обрамлены тегами example, endexample. Берем каждый абзац и выполняем split по началу каждого примера (тег ' example '). В итоге можем получить три вида кусков:
* "основной текст"
* "пример" + ' endexample '
* "пример" + ' endexample ' + "основной текст"

Учитывая то, что если подстрока, по которой производится деление, стоит в конце или в начале, split все равно выдаст 2 элемента, можно легко выявлять куски первого типа и помечать их как paragraph.

Далее мы получаем куски, которые после split'а по подстроке ' endexample ' слева содержат пример, а справа — основной текст. При этом мы знаем, что далее за основным текстом был либо конец абзаца, либо начинался следующий пример.

Сделаем два допущения:
* текст начала параграфа, если это не пример, всегда считаем достаточно длинным, чтобы включить его в основной текст
* недостаточно длинный текст в конце параграфа, если он не является примером, включаем в пример

Дальнейшая логика в коде.

In [64]:
def check_sides(text):
    """Функция отделяет все знаки, не являющиеся кириллицей и стоящие через пробел от первого и последнего
    слова, написанного кириллицей, в строке"""
    text_copy = deepcopy(text)
    text = re.sub(' term | endterm | example | endexample ', '', text)
    end_example = re.search('(^[^а-яА-ЯЁё]+?)[а-яА-ЯЁё]', text)
    if end_example:
        end_example = end_example.group(1)
    else:
        end_example = ''
    start_example = re.search('\s[^а-яА-ЯЁё]+?$', text) 
    if start_example:
        start_example = start_example.group(0)
    else:
        start_example = ''
    length = len(text_copy)
    length_end = len(end_example)
    if length_end != 0:
        length_end -= 1
    length_start = len(start_example)
    text = text_copy[length_end: length - length_start]
    return end_example, start_example, text

In [65]:
def extract_examples(paragraph, n=2, m=1):
    """Функция отделяет примеры от основного текста. Два идущих подряд слова-примера включаются
    в основной текст. Если между словами-примерами есть 1 обычное слово, оно включается в примеры."""
    paragraph = re.sub('\n', ' ', paragraph)
    print(paragraph)
    parts = paragraph.split(' example ')
    splitted = []
    example_line = ''
    for index, part in enumerate(parts):
        if part == '':  # параграф начинался с примера и первый элемент получился пустой строкой
            continue
        part = part.split(' endexample ')
        if len(part) == 1:  # параграф начинался не с примера, вписываем основной текст
            # проверим пунктуацию в конце текста
            end_example, start_example, part[0] = check_sides(part[0])
            splitted.append((part[0], 'paragraph'))
            example_line += start_example
        else:
            example_line += part[0]
            example_words = sum(1 for i in word_tokenize(example_line) if check_word(i))  # сумма идущих подряд слов-примеров
            text_words = sum(1 for i in word_tokenize(part[1]) if check_word(i))  # считаем количество слов в основном тексте
            if text_words <= m:
                # добавляем основной текст в примеры, если его мало
                # как вариант, слов основного текста может не быть вообще, но в переменной будет лежать пунктуация
                example_line += part[1]
            elif (example_words > n)and(text_words > m):
                # и примеров, и основного текста много, примеры записываем как примеры, текст — как текст
                # проверим пунктуацию по краям основного текста, потому что теги примеров обычно оставляют ее снаружи
                end_example, start_example, part[1] = check_sides(part[1])
                splitted.append((example_line + end_example, 'example'))
                example_line = start_example   
                splitted.append((part[1], 'paragraph'))
            elif (example_words <= n)and(text_words > m):
                # примеров мало, основного текста много, все записываем как текст
                # проверим пунктуацию в конце текста
                end_example, start_example, part[1] = check_sides(part[1])
                text = ' example ' + example_line + ' endexample ' + end_example + part[1]
                example_line = start_example  
                splitted.append((text, 'paragraph'))

    if example_line != '':  # если не обнулены примеры, вписываем
        splitted.append((example_line, 'example'))

    return splitted

In [66]:
def preprocess(text):
    """Токенизация, морфологический анализ библиотекой Natasha, присваивание тегов."""                
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    new_text =[]
    flag = ''
    for info in doc.tokens:
        word = info.text
        pos = info.pos
        
        if word not in ['example', 'endexample', 'term', 'endterm']: # термины и примеры определяются по тегам
            pass
        elif word == 'example':
            flag = 'example'
            continue
        elif word == 'endexample':
            flag = ''
            continue
        elif word == 'term':
            flag == 'term'
            continue
        elif word == 'endterm':
            flag == ''
            continue

        if (flag == '')and(check_word(word)):
            word = (word, 'word', pos, a.put_stress(word))
        elif (flag == '')and(check_word(word, lang='en')):
            word = (word, 'f_word', pos)
        elif (flag == 'example')and(check_word(word)):
            word = (word, 'example', pos, a.put_stress(word))
        elif (flag == 'term')and(check_word(word)):
            word = (word, 'term', pos, a.put_stress(word))
        else:
            word = (word, 'punct', pos)

        new_text.append(word)

    return new_text

* example - пример
* term - термин
* word - слово
* f_word - слово, содержащее латинские буквы, но не кириллицу (не термин и не пример)
* punct - пунктуация, цифры, приписанные не буквами

Соединим некоторые служебные слова (отрицательные частицы и непроизводные предлоги) со словами, за которыми они следуют.

In [67]:
def add_functional_words(paragraph):
    """Склеивание функциональных слов с теми, что идут после них"""
    functional_words = ['ни', 'не', 'в', 'с', 'со', 'к', 'за', 'из-под', 'из-за', 'по-над', 'у', 'над',
                        'на', 'под', 'над', 'без', 'до', 'о', 'об', 'от', 'при', 'для', 'из', 'по',
                        'Ни', 'Не', 'В', 'С', 'Со', 'К', 'За', 'Из-под', 'Из-за', 'По-над', 'У', 'Над',
                        'На', 'Под', 'Над', 'Без', 'До', 'О', 'Об', 'От', 'При', 'Для', 'Из', 'По']
    new_paragraph = []
    add = []
    for word in paragraph:
        if word[0] in functional_words:
            add.append(word)
            continue
        if add and word[1] in ['word', 'term', 'example']:
            add = [w[0] for w in add]
            text = ' '.join(add) + ' ' + word[0]
            text_stressed = ' '.join(add) + ' ' + word[3]
            word = (text, word[1], word[2], text_stressed)
            new_paragraph.append(word)
            add = []
        else:
            new_paragraph.extend(add)
            add = []
            new_paragraph.append(word)
    return new_paragraph

In [68]:
def final_preprocess(text):
    """Функция реализует всю описанную выше предобработку текста."""
    text = extract_examples(text)
    new_text = []
    paragraph = []
    for part in text:
        if part[1] == 'paragraph':
            paragraph.extend(preprocess(part[0]))
        else:
            paragraph = add_functional_words(paragraph)
            new_text.append((paragraph, 'paragraph'))
            paragraph = []
            new_text.append(part)
    if paragraph != []:
        paragraph = add_functional_words(paragraph)
        new_text.append((paragraph, 'paragraph'))
    return new_text

# Подсчет релевантных значений для слов и строк

In [69]:
def count(word):
    """Подсчет ударного слога и количества слогов в слове."""
    if word[1] not in ['example', 'term', 'word']:
        return 0, 0
    word = word[3]
    functional_words = [ 'ну', 'и', 'а', 'но', 'же']
    vowels = 'иёуеыаоэяю'
    if word in functional_words: #если слово служебное, в нем не считается ударение
        stressed = 0
    else:
        stressed = word.split("'")[0]
        stressed = sum(1 for c in stressed if c in vowels)
    syllables = sum(1 for c in word if c in vowels)
    return stressed, syllables

In [70]:
def count_line(line):
    """Подсчет ударных слогов и количества слогов в строке."""
    stresses_line = []
    syllables_line = 0
    indexes = []
    for index, word in enumerate(line):
        if word[1] in ['word', 'term', 'example']:
            stressed, syllables = count(word)
            if stressed != 0:
                stresses_line.append(syllables_line + stressed)
                indexes.append(index)
            syllables_line += syllables
    return stresses_line, syllables_line, indexes

# Проверки строки

In [71]:
def syllables_in_line(syllables_line):
    """Проверяет количество слогов в сформированной строке"""
    if syllables_line < 12:#если недостаточное количество слогов, продолжим набирать слова в строку
        return syllables_line, 'too few syllables'
    elif syllables_line > 17:#если слишком много слогов, меняем строку
        return syllables_line, 'too many syllables'
    else:
        return syllables_line, 'syllables true'

In [72]:
def first_stress(stresses_line):
    """Проверяет постановку первого ударения в сформированной строке."""
    if stresses_line[0] == 2:
        return stresses_line[0], 'the first stress does not suit'
    else:
        return stresses_line[0], 'first stress true'

In [73]:
def last_stress(stresses_line, syllables_line):
    """Проверяет постановку последнего ударения в сформированной строке"""
    if (syllables_line - stresses_line[-1]) != 1:#если последнее ударение не на предпоследний слог, меняем строку
        return (syllables_line - stresses_line[-1]), 'the last stress does not suit'
    else:
        return (syllables_line - stresses_line[-1]), 'last stress true'

In [74]:
def stresses(stresses_line, syllables_line, indexes_words):
    """Проверяет количество ударных долей в строке и расстояние между ними"""
    if stresses_line[0] == 1:#в зависимости от наличия пропущенного ударного слога в начале, присваеваем кол-во слогов
        steps = 1 #steps - ударные слоги, в том числе пропущенные
    elif stresses_line[0] in [2, 3, 4]:
        steps = 2
    elif stresses_line[0] in [5, 6, 7]:
        steps = 3
    elif stresses_line[0] in [8, 9, 10]:
        steps = 4
    checker = False #инициализируем значение флага: правильность кол-ва слогов между ударными слогами
    indexes_words = indexes_words[1:]  # обрезаем индекс слова с первым ударением в строке
    indexes_words.reverse()  # переворачиваем список, так как будем считать с конца
    for index, num in enumerate(zip(stresses_line[-1:0:-1], indexes_words)):#перебираем индексы ударных слогов от последнего до второго включительно
        index = (index + 1)*-1 #index counted from end
        step = num[0] - stresses_line[index -1] - 1 #количество слогов между текущим ударным слогом и предыдущим
        if step in [1, 2, 3, 4, 5, 6, 7, 8]:
            checker = True
            if step in [1, 2]:
                steps += 1
            elif step in [3, 4, 5]:#в зависимости от наличия "пропущенного" ударного слога, присваеваем кол-во слогов
                steps += 2
            elif step in [6, 7, 8]:
                steps += 3
        else:
            checker = False
            # между текущим ударением и предыдущим неправильное кол-во слогов
            # возвращаем индекс слова (!)
            return num[1], 'wrong number of syllables between stresses'

    if (checker == True)and(steps == 6):
        return 0, 'true'
    elif (checker == True)and(steps != 6):
        return steps, 'wrong number of stresses'

In [75]:
def check_line(stresses_line, syllables_line, indexes):
    """Общая проверка по всем критериям"""
    messages = []
    num, message = syllables_in_line(syllables_line)
    if message != 'syllables true':
        messages.append(message)
    num, message = first_stress(stresses_line)
    if message != 'first stress true':
        messages.append(message)
    num, message = last_stress(stresses_line, syllables_line)
    if message != 'last stress true':
        messages.append(message)
    num, message = stresses(stresses_line, syllables_line, indexes)
    if message != 'true':
        messages.append(message)
    if messages == []:
        return 'true'
    else:
        return messages

# Операция замены и работа с сущиствительными на -ние

Когда оригинальные слова не подходят для формирования строки, пытаемся добавить сокращенные версии существительных на -ние и найти синонимы для замены. Если подать в поиск синонимов термин, а не слово, функция попытается найти короткую форму слова на -ние, но не станет подключать семантическую модель. 

При поиске синонимов учитываются следующие факторы:

* род существительных в единственном числе при замене должен совпадать
* можно брать только первые два слова, не считая оригинального, предложенных моделью для замены
* части речи должны совпадать
* слово не является числительным
* слово не является местоимением или служебной частью речи

Дело в том, что с возрастанием косинусного расстояния сильно страдает смысл, а "сэкономить" слоги мы можем, заменив несколько слов в строке.

Далее, каждый синоним нужно поставить в нужную форму. Если поставить в нужную форму не получается, считается, что синоним не найден.

!Важно: прежде мы "приклеивали" предлоги и отрицательные частицы к словам. Необходимо учитывать это при замене.

In [76]:
def change_word(word):
    """Функция предлагает варианты замены, поставленные в нужную грамматическую форму.
    Числительные не заменяются."""
    word, mark, pos, stressed_word = word
    change = []
    
    if pos == 'NUM':
        return change
    word = word.split(' ')
    if len(word) != 1:
        func = ' '.join(word[:len(word) - 1]) + ' '
        word = word[-1]
    else:
        func = ''
        word = word[0]
    
    if pos == 'NOUN':
        if word[-3:] == 'ние':
            w = func + word[:-3] + 'нье'
            change.append(w)
        elif word[-3:] == 'ния':
            w = func + word[:-3] + 'нья'
            change.append(w)
        elif word[-3:] == 'нию':
            w = func + word[:-3] + 'нью'
            change.append(w)
        elif word[-4:] == 'нием':
            w = func + word[:-4] + 'ньем'
            change.append(w)
        elif word[-3:] == 'нии':
            w = func + word[:-3] + 'ньи'
            change.append(w)
        elif word[-4:] == 'ниям':
            w = func + word[:-4] + 'ньям'
            change.append(w)
        elif word[-4:] == 'ниях':
            w = func + word[:-4] + 'ньях'
            change.append(w)
        elif word[-5:] == 'ниями':
            w = func + word[:-5] + 'ньями'
            change.append(w)
    if mark == 'term':
        return change
    
    parsed_word = morph.parse(word)[0]
    tags = parsed_word.tag
    normal_form = parsed_word.normal_form + '_' + pos
    if normal_form in model.vocab:
        vector = model[normal_form]
    else:
        return change
    synonyms = model.most_similar(positive=[vector], negative=[], topn=5)
    synonyms = [i[0].split('_') for i in synonyms[1:]]
    for synonym, pos_synonym in synonyms:
        if pos == pos_synonym:
            if pos == 'NOUN':
                number = check_number(tags)
                gender = check_gender(tags)
                gender_synonym = check_gender(morph.parse(synonym)[0].tag)
                if (number == 'sing')and(gender != gender_synonym):
                    continue
            if pos not in ['ADV', 'ADP', 'CCONJ', 'SCONJ', 'INTJ', 'PART']: # неизменяемые части речи
                synonym = flection(synonym, tags)
            if synonym:
                change.append(func + synonym)
    return change

In [77]:
def flection(lex_neighb, tags):
    """Функция ставит лемму в нужную грамматическую форму.
    Ресурс: https://github.com/nevmenandr/word2vec-russian-novels"""
    tags = str(tags)
    tags = re.sub(',[AGQSPMa-z-]+? ', ',', tags)
    tags = tags.replace("impf,", "")
    tags = re.sub('([A-Z]) (plur|masc|femn|neut|inan)', '\\1,\\2', tags)
    tags = tags.replace("Impe neut", "")
    tags = tags.split(',')
    tags_clean = []
    for t in tags:
        if t:
            if ' ' in t:
                t1, t2 = t.split(' ')
                t = t2
            tags_clean.append(t)
    tags = frozenset(tags_clean)
    prep_for_gen = morph.parse(lex_neighb)
    ana_array = []
    for ana in prep_for_gen:
        if ana.normal_form == lex_neighb:
            ana_array.append(ana)
    for ana in ana_array:
        try:
            flect = ana.inflect(tags)
        except:
            print(tags)
            return None
        if flect:
            word_to_replace = flect.word
            return word_to_replace
    return None

In [78]:
def add_endings(target_words):
    """Отдельная функция модификации существительных на -ние
    для исправления последнего ударения в строке"""
    additional = []
    for num, word in target_words:
        if word[2] == 'NOUN' and word[0][-3:] == 'ние':
            w = word[0][:-3] + 'нье'
            w_s = putting_stress(w)
            w = (w, word[1], word[2], w_s)
            stressed, syllables = count(w)
            if syllables - stressed == 1:
                additional.append((num, w))
        elif word[2] == 'NOUN' and word[0][-3:] == 'ния':
            w = word[0][:-3] + 'нья'
            w_s = putting_stress(w)
            w = (w, word[1], word[2], w_s)
            stressed, syllables = count(w)
            if syllables - stressed == 1:
                additional.append((num, w))
        elif word[2] == 'NOUN' and word[0][-3:] == 'нию':
            w = word[0][:-3] + 'нью'
            w_s = putting_stress(w)
            w = (w, word[1], word[2], w_s)
            stressed, syllables = count(w)
            if syllables - stressed == 1:
                additional.append((num, w))
        elif word[2] == 'NOUN' and word[0][-4:] == 'нием':
            w = word[0][:-4] + 'ньем'
            w_s = putting_stress(w)
            w = (w, word[1], word[2], w_s)
            stressed, syllables = count(w)
            if syllables - stressed == 1:
                additional.append((num, w)) 
        elif word[2] == 'NOUN' and word[0][-3:] == 'нии':
            w = word[0][:-3] + 'ньи'
            w_s = putting_stress(w)
            w = (w, word[1], word[2], w_s)
            stressed, syllables = count(w)
            if syllables - stressed == 1:
                additional.append((num, w))
        elif word[2] == 'NOUN' and word[0][-4:] == 'ниям':
            w = word[0][:-4] + 'ньям'
            w_s = putting_stress(w)
            w = (w, word[1], word[2], w_s)
            stressed, syllables = count(w)
            if syllables - stressed == 1:
                additional.append((num, w))
        elif word[2] == 'NOUN' and word[0][-4:] == 'ниях':
            w = word[0][:-4] + 'ньях'
            w_s = putting_stress(w)
            w = (w, word[1], word[2], w_s)
            stressed, syllables = count(w)
            if syllables - stressed == 1:
                additional.append((num, w))
    return additional

# Типы ошибок:

### too few syllables

Исправление: Продолжаем собирать слова.

Проблема 1: далее следует кусок с примерами / конец текста.

Проблема 2: конец параграфа. Стоит ли оставлять деление на абзацы или объединить их, чтобы снизить количество таких ошибок?

### too many syllables: 

Такая ситуация возникает в случае, когда в строке n слов, и на n - 1 слове минимальное количество слогов (12) не было набрано, а на n-ом слове превысило лимит (17). Это значит, что последнее слово оказалось слишком длинным (минимум 7 слогов).

Исправление: укорачивать строку будем с помощью перестановки и/или замены. Перестановка: Попробуем поменять местами последнее слово со следующим, предполагая, что раз последнее слово оказалось достаточно длинным, следующее таким не окажется. Делать это будем только в том случае, если:

* между словами нет пунктуации
* текущее слово не пример
* следующее слово не пример
* следующее слово больше 1 слога (попытаемся избежать перемешивания союзов, непроизводных предлогов)
* следующее слово короче по количеству слогов
* разница между количеством слогов в нынешнем и следующем словах находится в промежутке \[k, k + 6\), где k — количество лишних слогов

Если этот метод не помогает, переходим к замене. Попробуем подвергнуть преобразованию два варианта строки, без перестановки и с, но только в случае, если перестановка не получилась, так как разница в количестве слогов не попала в нужный интервал. Приоритет отдается оригинальной строке: если она после замены стала подходящей, строка с перестановкой не затрагивается.

Логичнее всего заменять самое длинное слово, однако оно не должно быть примером. Составим список таких слов, упорядоченных по количеству слогов, по убыванию. Сохраним с каждым словом индекс в оригинальной строке. Посчитаем количество слогов, от которого нам нужно избавиться: s - 17, где s — количество слогов в текущей строке.

К каждому слову пытаемся подобрать синонимы (в случае терминов – сокращенные существительные на -ние). Происходит перебор упорядоченных ранее слов, внутри цикла — перебор их синонимов. Каждый раз, когда находится более короткое слово, пересчитывается число лишних слогов. Как только число лишних слогов меньше или равно нулю, а общее количество слогов в строке не меньше 12, строка считается правильной.

Если ничего не поможет, вернется оригинальная строка.

### syllables true:

!Важно: прежде мы сделали допущение, что в тексте могут присутствовать примеры, не более двух подряд. Если на конце сформированной по длине строки есть пример и следующее слово тоже является примером, вмещаем их в одну строку. Если длина строки будет слишком большой, проблема будет решаться функцией too_many_syllables методом замены. Помимо этого, в строку добавляются знаки препинания.

In [87]:
def too_many_syllables(line, num, num_syllables, preprocessed_text):
    copy_line = deepcopy(line)
    redundant = num_syllables - 17
    if num + 1 != len(preprocessed_text):  # если текущее слово оказалось последним в подаваемом тексте
        last_word = preprocessed_text[num]
        next_word = preprocessed_text[num + 1]
        length = count(last_word)[1]
        length_next = count(next_word)[1]
        dif = length - length_next
        if (last_word[1] != 'example')and(next_word[1] != 'example')and(length_next > 1)and(redundant <= dif < redundant+6):
            last_word = line[-1]
            line[-1] = next_word
            preprocessed_text[num + 1] = last_word
            # перестановка удалась
            return line
        elif (last_word[1] != 'example')and(next_word[1] != 'example')and(length_next > 1):
            alternative = deepcopy(line)
            alternative[-1] = next_word
            choose_line = [line, alternative]
        else:
            choose_line = [line,]
    else:
        choose_line = [line,]
                    
    for i, line in enumerate(choose_line):
        redundant = num_syllables - 17
        target_words = [(w, count(w)[1], num) for num, w in enumerate(line) if w[1] in ['word', 'term']]
        target_words = sorted(target_words, key=lambda x:x[1], reverse=True)
        for w, length, index in target_words:
            synonyms = change_word(w)
            if synonyms:  # синонимы могут не найтись
                synonyms = [(s, count(s)[1]) for s in synonyms]
                min_synonym = sorted(synonyms, key=lambda x:x[1])[0]
                dif = length - min_synonym[1]
                if dif > 0:
                    redundant -= dif
                    line[index] = (min_synonym[0], w[1], w[2], putting_stress(min_synonym[0]))
                if redundant <= 0:
                    break
        if (11 < redundant <= 0)and(i == 0):
            # замена в оригинальной строке
            return line
        elif (11 < redundant <= 0)and(i == 1):
            preprocessed_text[num + 1] = last_word
            # замена в строке с перестановкой
            return line
    # возврат оригинальной строки при неудаче
    return copy_line 

### the first stress does not suit

Исправление: выделяем кусок строки, отсеченный первым знаком пунктуации внутри текста. Находим первое по порядку слово, в котором ударение падает не на 2 слог. Передвигаем его в самое начало. Если передвигаемое слово является примером и за ним следует еще один пример, двигаем их вместе.

Если подходящих слов не нашлось, переходим к замене. Составляем список слов на замену: все они должны быть с пометкой word. Для каждого кандидата подбираем возможные синонимы. Перебираем синонимы и проверяем на два условия:

* ударение падает не на второй слог
* общее количество слогов в строке находится в диапазоне \[12, 17\]

Первый подошедший синоним подставляем в начало строки, удалив оригинальное слово.

В случае, если ничего не поможет, вернется оригинальная строки.

In [80]:
def the_first_stress_does_not_suit(line, syllables_line):
    line_check = deepcopy(line)
    indexes = [num for num, w in enumerate(line) if w[1] != 'punct']
    first_index = indexes[0]
    last_index = indexes[-1]
    start_line = line[:first_index]
    end_line = line[last_index:]
    line = line[first_index:last_index]
    first_punct = [num for num, w in enumerate(line) if w[1] == 'punct']
    if first_punct:
        target_words = [(num, w) for num, w in enumerate(line) if count(w)[0] != 2 and num < first_punct[0]]
    else:
        target_words = [(num, w) for num, w in enumerate(line) if count(w)[0] != 2]
    if target_words:
        # учитываем, что до этого мы сделали допущение: внутри текста могут идти только !!!два!!! слова-примера подряд
        target_index, target_word = target_words[0]
        if target_word[1] == 'example' and line[target_index + 1][1] == 'example':
            words_to_pop = [(target_index + 1, line[target_index + 1]), (target_index, target_word)]
        else:
            words_to_pop = [(target_index, target_word),]
        [line.pop(index) for index, w in words_to_pop]
        line.reverse()
        [line.append(w) for index, w in words_to_pop]
        line.reverse()
        return start_line + line + end_line
    else:
        if first_punct:
            target_words = [(num, w, change_word(w)) for num, w in enumerate(line) if w[1] == 'word' and num < first_punct[0]]
        else:
            target_words = [(num, w, change_word(w)) for num, w in enumerate(line) if w[1] == 'word']
        for index, word, synonyms in target_words:
            syllables_word = count(word)[1]
            for synonym in synonyms:
                stressed_synonym = a.put_stress(synonym)
                stressed, syllables_synonym = count((synonym, 'word', 'pos', stressed_synonym))
                if stressed != 2 and 12 <= (syllables_line - syllables_word + syllables_synonym) <= 17:
                    line.pop(index)
                    line.reverse()
                    line.append((synonym, word[1], word[2], putting_stress(synonym)))
                    line.reverse()
                    break
        line = start_line + line + end_line
        if line == line_check:
            # неудача
            return line
        else:
            return line

### the last stress does not suit
Исправление: выделяем кусок строки, отсеченный последним знаком пунктуации внутри строки. Находим первое с конца слово, ударение в котором падает на предпоследний слог, и двигаем его в конец. Если слово является примером и перед ним шел другой пример, передвигаем их вместе.

Если подходящих слов не нашлось, переходим к модификации существительных на -ние. В таких сокращенных словах ударение обычно падает как раз на предполедний слог, поэтому мы проходим этот шаг до работы с заменами. Составляем список слов, состоящий из терминов и слов основного текста, сохраняем при этом индексы. Проверяем, что от сокращения одного слога длина строки останется допустимой. Пытаемся заменить слова на -ние на сокращенные версии. 

Если модификация слов не получилась, собираем список слов на замену. Все они должны быть с пометкой word. Для каждого кандидата подбираем возможные синонимы. Перебираем синонимы и проверяем на два условия:

* ударение падает на предпоследний слог
* общее количество слогов в строке находится в диапазоне \[12, 17\]

Первый подошедший синоним подставляем в начало конец, удалив оригинальное слово.

В случае, если ничего не поможет, программа вернет оригинальную строку.

In [102]:
def the_last_stress_does_not_suit(line, index, syllables_line, preprocessed_text):
    line_check = deepcopy(line)
    indexes = [num for num, w in enumerate(line) if w[1] != 'punct']
    last_index = indexes[-1]
    if line[last_index][0] in [ 'ну', 'и', 'а', 'но', 'же']:
        diff = 0
        end_line = line[last_index:]
        line = line[:last_index]
    else:
        end_line = line[last_index + 1:]
        line = line[:last_index + 1]
    last_punct = [num for num, w in enumerate(line) if w[1] == 'punct']
    if last_punct:
        target_words = [(num, w) for num, w in enumerate(line) if (count(w)[1] - count(w)[0]) == 1 and count(w)[0] > 0 and last_punct[-1] < num]
    else:
        target_words = [(num, w) for num, w in enumerate(line) if (count(w)[1] - count(w)[0]) == 1 and count(w)[0] > 0]

    if not target_words and syllables_line - 1 >= 12:
        if last_punct:
            target_words = [(num, w) for num, w in enumerate(line) if w[1] in ['word', 'term'] and last_punct[-1] < num]
        else:
            target_words = [(num, w) for num, w in enumerate(line) if w[1] in ['word', 'term']]
        target_words = add_endings(target_words)
    n_words = []
    if (len(preprocessed_text) != index + 1 
        and line_check[-1][1] in ['word', 'term']):
        next_words = []
        for num, w in enumerate(preprocessed_text):
            if num > index and w[1] in ['term', 'word']:
                next_words.append((num, w))
            if w[1] == 'punct':
                break
        more_next_words = add_endings(next_words)
        next_words += more_next_words
        stress_orig, syllables_orig = count(line_check[-1])
        for num, w in next_words:
            stress_next, syllables_next = count(w)
            if syllables_next - stress_next == 1 and 12 <= syllables_line - syllables_orig + syllables_next <= 17:
                n_words.append((num, w))
                break
            
    if target_words:
        target_words.reverse()
        # учитываем, что до этого мы сделали допущение: внутри текста могут идти только !!!два!!! слова-примера подряд
        target_index, target_word = target_words[0]
        if target_word[1] == 'example' and line[target_index - 1][1] == 'example':
            words_to_pop = [ (target_index, target_word), (target_index - 1, line[target_index - 1])]
        else:
            words_to_pop = [(target_index, target_word),]
        [line.pop(index) for index, w in words_to_pop]
        words_to_pop.reverse()
        [line.append(w) for index, w in words_to_pop]
        return line + end_line
    elif n_words:
        w = n_words[0]
        preprocessed_text[w[0]] = line_check[-1]
        line_check[-1] = w[1]
        return line_check
    else:
        if last_punct:
            target_words = [(num, w, change_word(w)) for num, w in enumerate(line) if w[1] == 'word'and last_punct[-1] < num]
        else:
            target_words = [(num, w, change_word(w)) for num, w in enumerate(line) if w[1] == 'word']
        target_words.reverse()
        for index, word, synonyms in target_words:
            syllables_word = count(word)[1]
            for synonym in synonyms:
                stressed, syllables_synonym = count(synonym)
                if (syllables_synonym - stressed) == 1 and 12 <= (syllables_line - syllables_word + syllables_synonym) <= 17:
                    line.pop(index)
                    line.append((synonym, word[1], word[2], putting_stress(synonym)))
                    break
        line += end_line
        if line == line_check:
            # неудача
            return line
        else:
            return line

### 4 проверка, примерный подход

### wrong number of syllables between stresses

Функция, которая "поймала" ошибку, передала индекс слова, между ударным слогом которого и ударным слогом предыдущего слова неверное количество ударений. Программа допускает от одного до восьми безударных слогов между ударными, поэтому есть очень большая вероятность, что количество безударных слогов в таком случае равно нулю.

Исправление: делим строку на группы. Условия:
* группы разделяются знаками пунктуации
* первое и последнее слово не включаются в группы
* отсекаемые первым и последним словами знаки пунктуации не включаются в группы
* если в строке были примеры, идущие подряд, они оказываются объединены в один элемент
* каждый элемент имеет свой индекс, исходя из оригинального расположения слов в строке

Отсекаем все сформированные группы по индексу слова, на котором произошла ошибка. Если слово с этим индексом является первым в группе, группа считается несформированной.

Внутри каждой несформированной группы пораждаем все возможные комбинации слов. Генерируем строку из сформированных групп всеми возможными способами. Добавляем зафиксированные части: пунктуацию, первое и последнее слова. Каждый вариант строки проверяем на корректность. Если сформировалась полностью правильная строка, прерываем процесс и возвращаем ее. Параллельно сохраняем строки с ошибкой 'wrong number of stresses'.

Если полностью корректной строки не получилось, возвращаем строки с wrong number of stresses.

Если не получилось настроить расстояния, возвращаем оригинальную строку.

In [179]:
def wrong_number_of_syllables_between_stresses(index, line):
    new_line = []
    example = []
    for num, word in enumerate(line):
        if len(line) != num + 1 and word[1] == 'example' and line[num + 1][1] == 'example':
            example.append(word)
            continue
        if word[1] == 'example' and example:
            example.append(word)
            new_line.append((num - 1, example))
            example = []
        else:
            new_line.append((num, word))
    indexes = [num for num, w in new_line if w[1] != 'punct']
    first_index = indexes[0]
    last_index = indexes[-1]
    groups = []
    group = []
    for num, word in new_line:
        # не хотим затрагивать последнее слово
        if num == last_index:
            continue
        if word[1] != 'punct':
            group.append((num, word))
        if word[1] == 'punct' and group != []:
            groups.append(group)
            group = []
    if group != []:
        groups.append(group)
    # оставляем только группы, которые не были сформированы полностью
    groups = [group for group in groups if group[0][0] <= index]
    for num, group in enumerate(groups):
        groups[num] = list(permutations(group, len(group)))
    strings = list(product(*groups))
    true = []
    almost_true = []
    for num, string in enumerate(strings):
        new_string = []
        for i, g in enumerate(string):
            for w in g:
                new_string.append(w)
            if i + 1 != len(string):
                # вычленяем знаки пунктуации между группами
                last_group_index = max(g, key=lambda x:x[0])[0]
                first_group_index = min(string[i + 1], key=lambda x:x[0])[0]
                new_string.extend(new_line[last_group_index + 1:first_group_index])
        string = new_line[:first_index] + new_string + new_line[last_index:]
        s = []
        for word in string:
            if isinstance(word[1], list):
                s += word[1]
            else:
                s.append(word[1])
        stresses_line, syllables_line, indexes = count_line(s)
        info, message = stresses(stresses_line, syllables_line, indexes)
        stress, syllables = count(s[first_index])
        if message == 'true' and line[first_index] == s[first_index]:
            return 'true', s
        if message == 'true' and stress in [1, 3 ,4]:
            true.append(s)
        if message == 'wrong number of stresses' and stress in [1, 3 ,4]:
            almost_true.append((info, s))
    if true:
        return 'true', true[0]
    if almost_true:
        almost_true = sorted(almost_true, key=lambda x:x[0])
        return 'wrong number of stesses', almost_true[0][1]
    else:
        return 'original', line

### wrong number of stresses

Если получена эта ошибка, значит, все слова соответствуют назначенным критериям, кроме количества слогов в строке. Два случая:
* слогов меньше, чем нужно: можем осуществить вставку в конец так, чтобы добрать необходимое количество слогов. Последнее ударение должно падать на предпоследний слог
* слогов больше, чем нужно: поправить на этапе набора слов: невозможно использовать более 6 самостоятельных слов в строке? замена длинных слов на более короткие?

In [174]:
def set_capitals(orig_line, line):
    """Восстановление заглавных букв после перестановок"""
    for num, pair in enumerate(zip(orig_line, line)):
        orig, gen = pair
        gen = list(gen)
        if orig[0][0].istitle():
            if orig[2] != 'PROPN':
                gen[0] = gen[0].title()
            elif orig[2] == 'PROPN' and num != 0 and orig_line[num - 1][0] in ['.?!']:
                gen[0] = gen[0].title()
        else:
            if gen[0][0].istitle() and gen[2] != 'PROPN':
                gen[0] = gen[0].lower()
        line[num] = tuple(gen)
    return line

In [175]:
def rewrite(preprocessed_text):
    text_length = len(preprocessed_text)
    lines = []
    line = []
    for index, word in enumerate(preprocessed_text):
        line.append(word)
            
        stresses_line, syllables_line, indexes = count_line(line)
        # функция, поймавшая ошибку, передала информацию о количестве слогов в строке
        num_syllables, message = syllables_in_line(syllables_line)  # проверка 1/4
        if message == 'too few syllables':
            if index + 1 == len(preprocessed_text):
                lines.append((line, 'too few syllables'))
                return lines
            continue
        elif message == 'too many syllables':
            #print(message, [w[0] for w in line])
            line = too_many_syllables(line, index, num_syllables, preprocessed_text)
        elif message == 'syllables true':
            #print(message, [w[0] for w in line])
            if text_length != index + 1:
                if preprocessed_text[index][1] == 'example' and preprocessed_text[index + 1][1] == 'example':
                #print('more examples')
                    continue
                if preprocessed_text[index + 1][1] == 'punct':
                    continue
        
        copy_line = deepcopy(line)
        
        stresses_line, syllables_line, indexes = count_line(line)
        # функция, поймавшая ошибку, передала информацию о расположении последнего ударения
        stress, message = last_stress(stresses_line, syllables_line) # проверка 3/4
        if message == 'the last stress does not suit':
            #print(message, [w[0] for w in line])
            line = the_last_stress_does_not_suit(line, index, syllables_line, preprocessed_text)
            #print([w[0] for w in line])
        elif message == 'last stress true':
            #print(message, [w[0] for w in line])
            pass
        
        stresses_line, syllables_line, indexes = count_line(line)
        # функция, поймавшая ошибку, передала информацию о расположении первого ударения
        stress, message = first_stress(stresses_line) # проверка 2/4
        if message == 'the first stress does not suit':
            #print(message, [w[0] for w in line])
            line = the_first_stress_does_not_suit(line, syllables_line)
            #print([w[0] for w in line])
        elif message == 'first stress true':
            #print([w[0] for w in line])
            pass
        
        stresses_line, syllables_line, indexes = count_line(line)
        info, message = stresses(stresses_line, syllables_line, indexes)
        if message == 'wrong number of syllables between stresses':
            # функция, поймавшая ошибку, передала индекс слова, между которым и следующим неправильное количество слогов
            #print(message, [w[0] for w in line])
            message, line = wrong_number_of_syllables_between_stresses(index, line)
        stresses_line, syllables_line, indexes = count_line(line)
        messages = check_line(stresses_line, syllables_line, indexes)
        if text_length != index + 1 and message != 'true':
            stressed, additional_syllables = count(preprocessed_text[index + 1])
            if num_syllables + additional_syllables <= 17:
                continue
        if message == 'wrong number of stresses':
            #print('FINAL', message, line)
            line = set_capitals(copy_line, line)
            lines.append((line, messages))
            line = []
        elif message == 'true':
            #print('FINAL', message, [w[0] for w in line])
            line = set_capitals(copy_line, line)
            lines.append((line, messages))
            line = []
        elif message == 'original':
            #print('FINAL', message, [w[0] for w in line])
            line = set_capitals(copy_line, line)
            lines.append((line, messages))
            line = []
    return lines

In [176]:
cl = '   § 1454. Итак, глаголы несовершенного вида употребляются в более широких и более разнообразных условиях, чем глаголы совершенного вида. Помимо тех типов ситуаций, которые могут передаваться (по разному) глаголами обоих видов, есть и такие типы ситуаций, которые могут быть переданы лишь при участии глаголов несовершенного вида.'

In [177]:
for t, title in final_preprocess(cl):
    if title == 'paragraph':
        lines = rewrite(t)
        for line, message in lines:
            print([w[0] for w in line])

   § 1454. Итак, глаголы несовершенного вида употребляются в более широких и более разнообразных условиях, чем глаголы совершенного вида. Помимо тех типов ситуаций, которые могут передаваться (по разному) глаголами обоих видов, есть и такие типы ситуаций, которые могут быть переданы лишь при участии глаголов несовершенного вида.
rrrrrrrrr [(2, ('.', 'punct', 'PUNCT'))]
rrrrrrrrr [(2, ('.', 'punct', 'PUNCT'))]
['Итак', ',', 'глаголы', 'несовершенного', 'вида']
['употребляются', 'в более', 'и', 'более', 'широких']
['разнообразных', 'условиях', ',', 'чем', 'глаголы']
['совершенного', 'вида', '.', 'Тех', 'помимо', 'типов']
['ситуаций', ',', 'которые', 'могут', 'передаваться', '(']
['по разному', ')', 'глаголами', 'обоих', 'видов', ',', 'есть', 'и']
['типы', 'такие', 'ситуаций', ',', 'которые', 'могут']
['быть', 'при участии', 'лишь', 'глаголов', 'переданы', 'вида']
['несовершенного', '.']


In [187]:
overall = 0
true = 0
too_few_syllables_error = 0
too_many_syllables_error = 0
the_last_stress_does_not_suit_error = 0
the_first_stress_does_not_suit_error = 0
wrong_number_of_syllables_between_stresses_error = 0
wrong_number_of_stresses_error = 0
for num, text in enumerate(clean_text):
    print(num)
    if text[1] == 'paragraph':
        new_text = final_preprocess(text[0])
    else:
        print(text[0], '\n')
        continue
    for part, title_b in new_text:
        if title_b == 'paragraph':
            lines = rewrite(part)
            text = ''
            for line, message in lines:
                if message == 'true':
                    true += 1
                if 'too few syllables' in message:
                    too_few_syllables_error += 1
                if 'too many syllables' in message:
                    too_many_syllables_error += 1
                if 'the last stress does not suit' in message:
                    the_last_stress_does_not_suit_error += 1
                if 'the first stress does not suit' in message:
                    the_first_stress_does_not_suit_error += 1
                if 'wrong number of syllables between stresses' in message:
                    wrong_number_of_syllables_between_stresses_error += 1
                if 'wrong number of stresses' in message:
                    wrong_number_of_stresses_error += 1
                overall += 1
                for word in line:
                    text += word[0] + ' '
                print(text, '\t', message, '\n')
                text = ''
        else:
            print(part + '\n')

0
УПОТРЕБЛЕНИЕ ВИДОВ 

1
Вступительные замечания 

2
   § 1437. Категориальные грамматические значения совершенного и несовершенного вида  реализуются в разных условиях контекста. В зависимости от этих условий различаются определенные типы употребления видов. Они группируются на основании типов ситуаций, которые передаются при участии глагольного вида. Выделяются четыре типа таких ситуаций.
Категориальные грамматические значенья  	 ['wrong number of stresses'] 

совершенного и несовершенного вида  	 true 

реализуются условиях в разных контекста .  	 true 

Различаются в зависимости от этих условий  	 ['wrong number of stresses'] 

определенные употребления типы  	 true 

видов . Они группируются на основаньи типов  	 true 

ситуаций , которые передаются  	 ['wrong number of stresses'] 

при участии глагольного вида . Выделяются  	 ['the last stress does not suit', 'wrong number of stresses'] 

четыре типа таких ситуаций .  	 too few syllables 

3
   1. Ситуация единичного (неповторяющ

В Упомянутых употребления типах видов  	 true 

и их разновидностях представлены значенья ,  	 ['the first stress does not suit', 'wrong number of stresses'] 

выражаемые глагольным в сочетании видом  	 ['wrong number of stresses'] 

с контекстом ( в части случаев - в сочетании также  	 ['the first stress does not suit', 'wrong number of stresses'] 

действия со способом глаголов и их  	 ['the last stress does not suit'] 

грамматическим значением ) . Таковы , например ,  	 ['the last stress does not suit', 'wrong number of stresses'] 

обусловленные семантические контекстом  	 true 

типами различия употребления между  	 true 

несовершенного вида , представленными в следующих  	 ['the last stress does not suit'] 

предложениях :  	 too few syllables 

Он читал, когда я вошел (

( употребление процессное конкретно ) ;  	 true 

Он читал по вечерам (

( неограниченно кратное употребленье ) ;  	 ['wrong number of stresses'] 

 - Ты читал эту книгу? - Читал (

( предположительное упрощен

Тип фактический употребления конкретно  	 true 

для совершенного основным является вида ,  	 true 

центральным . Он реализуется в самых разнообразных  	 ['the first stress does not suit', 'wrong number of stresses'] 

дискурса условиях . Для Его реализации  	 ['the last stress does not suit', 'wrong number of stresses'] 

минимальный контекст достаточен ( вплоть предложения  	 ['the last stress does not suit'] 

до нераспространенного :  	 too few syllables 

 - Замолчите!; Нашел!; Приняли?). 

Употребления достаточно для такого ,  	 true 

чтобы в контексте не было указаний  	 ['wrong number of stresses'] 

на повторяемость ( обычность , типичность ) ситуации .  	 ['the last stress does not suit', 'wrong number of stresses'] 

Ситуации поскольку такие при совершенном  	 ['wrong number of stresses'] 

виде возможны лишь ограниченных в особых  	 true 

условиях , совершенного глаголы  	 ['the first stress does not suit'] 

вида чаще всего выступают фактическом в конкретно  	 ['wrong n

Разных глаголы видов могут сочетаться  	 true 

друг с другом . В Таких различные случаях разновидности передаются  	 ['too many syllables', 'wrong number of stresses'] 

соотношения и целостных процессов  	 true 

фактов ; например , " процесс - наступление факта " :  	 true 

Однажды весною охотник проходил возле березового болота и вдруг услыхал громкое хлопанье крыльев (И. Соколов Микитов); "

" наступление факта - последующий процесс " :  	 ['the last stress does not suit'] 

Привалов поздоровался с девушкой и несколько мгновений смотрел на нее удивленными глазами (М. Сиб.).

18
   § 1440. Выделяются особые разновидности типов употребления видов, каждая из которых характеризуется дополнительными семантическими признаками. Особыми разновидностями конкретно фактического типа употребления совершенного вида являются потенциальная, перфектная, ограниченно длительная и суммарная. Разновидностями конкретно процессного типа употребления несовершенного вида являются конативная и подчеркнут

Разновидностью фактического конкретно  	 true 

употребления совершенного типа вида  	 true 

безналичного является выраженье  	 true 

результата ( перфектная разновидность ) . В Таком  	 ['the last stress does not suit'] 

могут выступать употреблении формы  	 true 

времени прошедшего , и деепричастий причастий :  	 ['wrong number of stresses'] 

Ты, наверно, озяб в своих танцевальных туфлях (Солоух.); [Лавров:] Городище будет затоплено (Кавер.); "Вот еще", - подумал Яков, разглядывая угловатую фигуру монаха, прислонившуюся к стволу березы, им же и посаженной (Горьк.); Прижав руки к груди, мать.. стояла у его постели (Горьк.).

23
   Способность выступать в перфектном употреблении - свойство главным образом глаголов совершенного вида: ограниченность целостного действия пределом - одна из предпосылок того, что внимание концентрируется не на самом осуществлении действия, а на его последствиях, результате. Для того чтобы был выражен наличный результат, вызвавшее его действие должно быт

...

Сообщается о действии ( обычно  	 true 

отношении и состоянии , но иногда  	 ['the last stress does not suit'] 

и о действии , связанном с определенным  	 ['the last stress does not suit', 'wrong number of stresses'] 

изменением , развитием ) , имеющем  	 ['the last stress does not suit'] 

неограниченную временную протяженность .  	 ['the last stress does not suit'] 

Это действие ( состояние , отношенье )  	 ['the first stress does not suit'] 

определенный охватывает собою  	 true 

времени период - настоящего , прошедшего или  	 ['wrong number of stresses'] 

будущего , оно которого как представлено постоянное в рамках  	 ['too many syllables', 'wrong number of stresses'] 

и непрерывное :  	 too few syllables 

 - Образа в доме держите для обмана (Горьк.); В комнате его завелись какие то ненужные, но красивенькие штучки, на стене висела вышитая бисером картина - девичий хоровод (Горьк.); Население устроилось жить возле больших рек (Пришв.); Поселок этот стоит у реки. Река течет

Случаях может быть в таких несущественным ,  	 true 

ли является действие единичным или  	 true 

повторяющимся , - важно то , что оно было .  	 ['wrong number of syllables between stresses'] 

Оказывается достаточным действия называнье  	 true 

безотносительно к тем иным особенностям или  	 ['wrong number of stresses'] 

темперамента его осуществления :  	 ['the last stress does not suit'] 

 - Да я не знаю, худо это или хорошо, только они виделись (Леск.); Теперь вот, недавно, пришлось посылать Илье денег куда то в Сибирь (Горьк.); Отец Анастасий.., забыв про свое решение уходить домой, опустился на стул (Чех.); Позвонил, чтобы давали машину (Герман); Завтра надо будет выплачивать сотрудникам жалованье (Катаев).

58
   В данном типе употребления грамматическое значение несовершенного вида проявляется не как противоположность значению совершенного вида, а как нейтральное отношение к признаку ограниченности действия пределом: действие обозначается безотносительно к его ограниченности

Вид при отрицании употребляется совершенный  	 ['wrong number of stresses'] 

случаях в тех , когда выражается предостереженье :  	 ['wrong number of stresses'] 

Не забудь!; Не опоздай!; Не упади!; Не ошибись!; Не потеряй!; Не простудись!

62
   При употреблении форм прошедшего времени также наблюдается подобное соотношение глаголов совершенного вида в утвердительной конструкции и несовершенного вида - в отрицательной: -  example Зачем endexample   example же endexample   example вы endexample   example набросились endexample   example на endexample   example меня endexample ? -  example Я endexample  -  example не endexample   example набрасывался endexample   example (Горьк.) endexample ; -  example А endexample   example на endexample   example сколько endexample   example же endexample   example ты endexample   example его endexample   example в endexample   example арестантскую endexample   example упек endexample ? -  example Я endexample   example его endexample   example не en

Разновидности к рассматриваемой также  	 true 

случаи относятся , когда при помощи  	 ['the last stress does not suit'] 

несовершенного глагола с отрицанием вида  	 ['wrong number of stresses'] 

выражается состояние , вызванное  	 ['the last stress does not suit', 'wrong number of stresses'] 

неосуществленного последствиями действия :  	 ['the last stress does not suit'] 

Было холодно, я три ночи не спал, измучился и начал сердиться (Лерм.); [Вожеватов:] Не выпьем ли холодненького, Мокий Парменыч? [Кнуров:] Что вы, утром то! Я еще не завтракал (А. Остр.); [Ольга Андреевна:] Ишь исхудала. С утра не ела (Кавер.). 

Употребление такое характерно  	 true 

для немногих глаголов , лексическое значенье  	 ['wrong number of stresses'] 

предполагает которых того возможность , что  	 ['the last stress does not suit'] 

действия отсутствие определенное вызывает  	 ['wrong number of stresses'] 

состояние ( голода , усталости и подобного ) .  	 ['the last stress does not suit', 'wrong numbe

Ситуация единичного ( неповторяющегося  	 ['too many syllables', 'the last stress does not suit'] 

) действия  	 too few syllables 

70
Конкретно фактический тип употребления. Разновидности: потенциальная, перфектная, ограниченно длительная, суммарная
Тип фактический употребления конкретно .  	 true 

Разновидности : потенциальная , перфектная ,  	 ['the last stress does not suit'] 

ограниченно длительная , суммарная  	 ['the last stress does not suit', 'wrong number of stresses'] 

71
Конкретно процессный тип употребления. Разновидности: конативная, подчеркнуто длительная
Тип конкретно употребления процессный .  	 true 

Разновидности : конативная , подчеркнуто  	 true 

длительная  	 too few syllables 

72
2.
2 .  	 too few syllables 

73
Ситуация повторяющегося действия
Ситуация повторяющегося действия  	 ['the last stress does not suit'] 

74
Наглядно примерный тип употребления. Разновидности: потенциальная, перфектная
Тип наглядно употребления примерный .  	 true 

Разновидности

In [185]:
#ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018
print(overall, 'total \n',
true, ' true \n',
too_few_syllables_error, ' too few syllables \n',
too_many_syllables_error, ' too many syllables \n',
the_last_stress_does_not_suit_error, ' the last stress does not suit \n',
the_first_stress_does_not_suit_error, ' the first stress does not suit \n',
wrong_number_of_syllables_between_stresses_error, ' wrong number of syllables between stresses \n',
wrong_number_of_stresses_error, ' wrong number of stresses ')

823 total 
 355  true 
 87  too few syllables 
 13  too many syllables 
 163  the last stress does not suit 
 36  the first stress does not suit 
 26  wrong number of syllables between stresses 
 260  wrong number of stresses 


In [181]:
#tayga_upos_skipgram_300_2_2019
print(overall, 'total \n',
true, ' true \n',
too_few_syllables_error, ' too few syllables \n',
too_many_syllables_error, ' too many syllables \n',
the_last_stress_does_not_suit_error, ' the last stress does not suit \n',
the_first_stress_does_not_suit_error, ' the first stress does not suit \n',
wrong_number_of_syllables_between_stresses_error, ' wrong number of syllables between stresses \n',
wrong_number_of_stresses_error, ' wrong number of stresses ')

822 total 
 359  true 
 87  too few syllables 
 14  too many syllables 
 160  the last stress does not suit 
 28  the first stress does not suit 
 26  wrong number of syllables between stresses 
 259  wrong number of stresses 


In [188]:
#araneum_upos_skipgram_300_2_2018.vec
print(overall, 'total \n',
true, ' true \n',
too_few_syllables_error, ' too few syllables \n',
too_many_syllables_error, ' too many syllables \n',
the_last_stress_does_not_suit_error, ' the last stress does not suit \n',
the_first_stress_does_not_suit_error, ' the first stress does not suit \n',
wrong_number_of_syllables_between_stresses_error, ' wrong number of syllables between stresses \n',
wrong_number_of_stresses_error, ' wrong number of stresses ')

821 total 
 358  true 
 85  too few syllables 
 13  too many syllables 
 163  the last stress does not suit 
 27  the first stress does not suit 
 26  wrong number of syllables between stresses 
 260  wrong number of stresses 


### Ошибки:

* неправильно расставленное ударение: гора/горе, не исправляется контекстуально

In [ ]:
a.put_stress('на высокой горе')